# 9. 아이유팬이 좋아할 만한 다른 아티스트 찾기

## 1) 데이터 준비와 전처리

In [158]:
#!mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
#!ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m
!ls


aiffel	data


In [211]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [212]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')


orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [213]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count1'}, inplace=True)

In [214]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies['title'] = movies['title'].str.lower() # 검색을 위해 소문자 변경
movies.head()


,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


In [215]:
title = movies['title'].values
movie_id = movies['movie_id'].values
movie_dic = {movie_id:title for movie_id ,title in zip(movie_id,title)}

In [216]:
ratings_movie_id = ratings['movie_id'].values
ratings_movie_id


array([1193,  661,  914, ...,  562, 1096, 1097])

In [217]:
ratings_movie_name = []
for i in ratings_movie_id:
    if i in movie_dic.keys():
        ratings_movie_name.append(movie_dic[i])

In [218]:
ratings_movie_name

["one flew over the cuckoo's nest (1975)",
 'james and the giant peach (1996)',
 'my fair lady (1964)',
 'erin brockovich (2000)',
 "bug's life, a (1998)",
 'princess bride, the (1987)',
 'ben-hur (1959)',
 'christmas story, a (1983)',
 'snow white and the seven dwarfs (1937)',
 'wizard of oz, the (1939)',
 'beauty and the beast (1991)',
 'gigi (1958)',
 'miracle on 34th street (1947)',
 "ferris bueller's day off (1986)",
 'sound of music, the (1965)',
 'airplane! (1980)',
 'tarzan (1999)',
 'bambi (1942)',
 'awakenings (1990)',
 'big (1988)',
 'pleasantville (1998)',
 'wallace & gromit: the best of aardman animation (1996)',
 'back to the future (1985)',
 "schindler's list (1993)",
 'meet joe black (1998)',
 'pocahontas (1995)',
 'e.t. the extra-terrestrial (1982)',
 'titanic (1997)',
 'ponette (1996)',
 'close shave, a (1995)',
 'antz (1998)',
 'girl, interrupted (1999)',
 'hercules (1997)',
 'aladdin (1992)',
 'mulan (1998)',
 'hunchback of notre dame, the (1996)',
 'last days of di

In [219]:
ratings['name'] = ratings_movie_name

In [220]:
del ratings['movie_id']
del ratings['timestamp']
ratings.head()

,user_id,count1,name
0,1,5,one flew over the cuckoo's nest (1975)
1,1,3,james and the giant peach (1996)
2,1,3,my fair lady (1964)
3,1,4,erin brockovich (2000)
4,1,5,"bug's life, a (1998)"


## 2) 데이터 분석

In [221]:
a = len(ratings['name'].unique()) # ratings에 있는 유니크한 영화 개수
b = len(ratings['user_id'].unique()) # rating에 있는 유니크한 사용자 수 
c = ratings.groupby('name')['user_id'].count()
print(f'ratings에 있는 유니크한 영화 개수 : {a} \nratings에 있는 유니크한 사용자 수 : {b}')
print('인기 많은 영화 수 정렬 :\n',c.sort_values(ascending=False).head(30))

ratings에 있는 유니크한 영화 개수 : 3628 
ratings에 있는 유니크한 사용자 수 : 6039
인기 많은 영화 수 정렬 :
 name
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                              

## 3) 내가 선호하는 영화 5가지 Rating 추가

In [222]:
my_favorite = ['toy story (1995)', 'Jumanji (1995)', 'matrix, the (1999)', 'terminator, the (1984)', 'back to the future (1985)']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'name': my_favorite, 'count1':[5]*5})

if not ratings.isin({'user_id':['6041']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다.
ratings.tail()

,user_id,count1,name
0,6041,5,toy story (1995)
1,6041,5,Jumanji (1995)
2,6041,5,"matrix, the (1999)"
3,6041,5,"terminator, the (1984)"
4,6041,5,back to the future (1985)


## 4) CSR matrix 만들기

In [224]:
movie_data = {v:k for k,v in enumerate(ratings['name'].unique())}
user_id_data = {v:k for k,v in enumerate(ratings['user_id'].unique())}

In [225]:
temp_user_data = ratings['user_id'].map(user_id_data.get).dropna()
if len(temp_user_data) == len(ratings):
    print('user_id_cloumn indexing OK!!')
    ratings['user_id'] = temp_user_data
else:
    print('user_id_cloumn indexing Fail !!')

temp_movie_ratings = ratings['name'].map(movie_data.get).dropna()
if len(temp_movie_ratings) == len(ratings):
    print('movie colmn indexing OK!')
    ratings['name'] = temp_movie_ratings
else:
    print('movie colmn indexing Fail!')



user_id_cloumn indexing OK!!
movie colmn indexing OK!


In [226]:
ratings

,user_id,count1,name
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
0,6039,5,40
1,6039,5,3628
2,6039,5,124
3,6039,5,200


In [227]:
# num_user = max(ratings['user_id'])+1
# num_movie = max(ratings['name'])+1
num_user = ratings['user_id'].nunique()
num_movie = ratings['name'].nunique()


csr_data = csr_matrix((ratings.count1, (ratings.user_id, ratings.name)), shape= (num_user, num_movie))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model 훈련

In [228]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [231]:
als_model = AlternatingLeastSquares(factors=1000, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)
csr_data_transpose = csr_data.T
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [233]:
myid, movie_name = user_id_data[6041], movie_data['Jumanji (1995)']
myid_vector, movie_vector = als_model.user_factors[myid], als_model.item_factors[movie_name]

In [234]:
np.dot(myid_vector,movie_vector)

0.035621665

In [235]:
movie1_vector = als_model.item_factors[movie_data['tarzan (1999)']]
np.dot(myid_vector,movie1_vector)

-0.0030035186

## 6) 내가 선호하는 영화와 비슷한 영화 추천

In [236]:
def get_similar_movie(movie_name: str): # 위 코드 함수화
    movie_id = movie_data[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [[i[0]] for i in similar_movie]
    return similar_movie

In [237]:
get_similar_movie('toy story (1995)')

['toy story (1995)',
 'Jumanji (1995)',
 'soft toilet seats (1999)',
 'slappy and the stinkers (1998)',
 'an unforgettable summer (1994)',
 'jerry & tom (1998)',
 'truce, the (1996)',
 'five wives, three secretaries and me (1998)',
 'circus (2000)',
 'talk of angels (1998)']

## 7) 내가 좋아할 만한 영화 추천

In [238]:
movie_re = als_model.recommend(myid,csr_data,N=20,filter_already_liked_items=True)

In [244]:
[idx_to_movie[i[0]] for i in movie_re]

['terminator 2: judgment day (1991)',
 'hard rain (1998)',
 'instinct (1999)',
 'robocop (1987)',
 'near dark (1987)',
 'die hard (1988)',
 'suddenly, last summer (1959)',
 'aliens (1986)',
 'forget paris (1995)',
 'shower (xizhao) (1999)',
 'blast from the past (1999)',
 'holiday inn (1942)',
 'tora! tora! tora! (1970)',
 'kramer vs. kramer (1979)',
 'lolita (1997)',
 'poison ivy (1992)',
 'brassed off (1996)',
 'south park: bigger, longer and uncut (1999)',
 'blade (1998)',
 'captain horatio hornblower (1951)']

# 회고 : 

- 처음하는 추천시스템이였는데 생각한 것 보다 좀 더 여러가지 경우를 배우게 된 것 같다. 처음 배우는 모델과 영화 별로 숫자와 인덱싱 하는 방법 CSR matrix를 상룔하는 방법까지 다양하게 응용할 수 있게 된 것 같다. 아무래도 이런것들을 다른데에 적용하면서 배우는게 의미가 더 있을듯싶다..